# Model evaluation

In the paper, we consider a classification problem where inputs to the model are questions $x$ paired with candidate answers $y$ to constitute concatenated sequences.
The generative model then processes these concatenated question-answer pairs to predict the most probable answer $\hat{y}$ from the provided choices $Y$ for a given $x$:
\begin{align*}
\hat{y} = \underset{y \in Y}{\text{arg max }} p_{\text{LM}}(y|x).
\end{align*}
Here, the probability of the token sequence
$y$ is derived as the product of individual token $y_{[i]}$ probabilities within the sequence, conditioned on
$x$ and the preceding tokens $y_{[1:i-1]}$:
\begin{align*}
p_{\text{LM}}(y|x) = \prod_{i=1}^{|y|} p_{\text{LM}}(y_{[i]}|x, y_{[1:i-1]}),
\end{align*}
where $|y|$ is the number of tokens composing the answer $y$.

For the entailment generation benchmarks, we use texts concatenated with possible completions as inputs to the model.
We compare the quantized and full-precision models with the difference in the probabilities of the sequences  $p_{\text{LM}}(y|x)$, further referred to as confidences.

To compute the scores $\hat{y}$, we use lm-evaluation harness framework and detailed output for each evaluation obtained with `write_out` argument: https://github.com/EleutherAI/lm-evaluation-harness.

*Note that while we use the December 2023 version of the framework, you can use instead the current version (master branch) and replace the arguments with current arguments:*
```
!lm_eval --model hf \
    --model_args pretrained=model-name-or-path,autogptq=model.safetensors,gptq_use_triton=True \
    --tasks hellaswag

```
* `write_out` was replaced with `log_samples` argument.

In [1]:
!pip install auto-gptq==0.7.1 torch==2.3.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
goog

In [1]:
# !git clone https://github.com/EleutherAI/lm-evaluation-harness.git
%cd lm-evaluation-harness
!git checkout "add-siqa"
!pip install -e . -q

/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/root/quantized-lm-confidence/notebooks/lm-evaluation-harness
Already on 'add-siqa'
Your branch is up to date with 'origin/add-siqa'.
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
# !export LC_ALL="en_US.UTF-8"
# !export LD_LIBRARY_PATH="/usr/lib64-nvidia"
# !export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
# !ldconfig /usr/lib64-nvidia

In [5]:
#@title Model type and tokenizer
model_path="bigscience/bloom-1b7"#@param {type:"string"}
tokenizer_path='bigscience/bloom-1b7'#@param {type:"string"}

In [6]:
output_base_path=model_path
output_path=output_base_path+"_suite.json"

In [9]:
!python main.py \
    --model hf-causal-experimental \
    --model_args pretrained=$model_path,tokenizer=$tokenizer_path \
    --device cuda:0 \
    --tasks hellaswag \
    --write_out \
    --no_cache \
    --output_path $output_path \
    --output_base_path $output_base_path

Selected Tasks: ['hellaswag']
Task: hellaswag; number of docs: 10042
Task: hellaswag; document 0; context prompt (starting on next line):
Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.
(end of prompt on previous line)
Requests: [Req_loglikelihood('Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.', ' You can visit a lingerie shop and have them measure you to help you fit a bra to your size, or measure yourself before you shop for a new bra to ensure that you get a good fit. Use a flexible tape measure, like one found in a sewing kit.')[0]
, Req_loglikelihood('Personal Care and Style: How to increase breast size with a bra. Check your bra size. Weari

For non-quantized models, remove `quantized` and `gptq_use_triton` arguments.